## Replacing missing values with expressions
By the end of this lecture you will be able to:
- replace missing values with an expression on the same column
- replace missing values based on other columns
- replace missing values based on a condition
- replace missing values with interpolation

In [ ]:
import polars as pl

We again create a simple `DataFrame` for this lecture

In [ ]:
df = pl.DataFrame(
    {
        'col1':[0,None,2,3],
        "col2":[0,None,None,3],
        "col3":[4,5,6,7]
    }
)
df

We are not limited to the built-in strategies as we can also use expressions to replace missing values.

### Replace missing values using an expression from the same column
For example we can replace the missing values with the median of the non-`null` values for that column

In [ ]:
(
    df
    .with_columns(
        pl.col("col1").fill_null(pl.median("col1")).name.suffix("_new"),
    )
)


### Interpolation
We can replace missing values with linear interpolation

In [ ]:
(
    df
    .with_columns(
        pl.all().interpolate().name.suffix("_new"),
    )
)

### Replace missing values using a different column

We can also replace missing values with the values from another column

In [ ]:
(
    df
    .with_columns(
        pl.col("col2").fill_null(pl.col("col3")).name.suffix("_new"),
    )
)

## Replacing missing values based on a sequence of columns
We can replace missing values based on a sequence of columns with `coalesce`.

In this example we have 3 columns `a`,`b` and `c`

In [ ]:
df_coalesce = pl.DataFrame(
    [
        {'a': None, 'b': 1.0, 'c': 1.0},
        {'a': None, 'b': 2.0, 'c': 2.0},
        {'a': None, 'b': None, 'c': 3.0},
        {'a': None, 'b': None, 'c': None}
    ]
)
df_coalesce

We want to create a new column that has the first non-`null` value as we go through a sequence of columns in order. 

We do this with `pl.coalesce` where we can also specify a fill value if all of the columns are `null`

In [ ]:
(
    df_coalesce
    .with_columns(
        pl.coalesce(["a", "b", "c", 9.0]).alias("d")
    )
)

## Exercises
In the exercises you will develop your understanding of:
- replacing missing values with an expression
- replacing missing values with interpolation
- replacing missing values with `coalesce`

### Exercise 1
Replace `null` values in the `Age` column to have the `median` of the `Age` column

In [ ]:
csv_file = "../data/titanic.csv"
(
    pl.read_csv(csv_file)
   <blank>
)

Replace `null` values in the `Age` column to have the `median` of the `Age` column **based on whether the passenger is `male` or `female` in the `Sex` column**.

Expand the following cell if you want a hint

In [ ]:
#Hint: in each fill_null call you need to apply a filter to the `Sex` column before you can call median

In [ ]:
csv_file = "../data/titanic.csv"
(
    pl.read_csv(csv_file)
    .with_columns(
        <blank>
        .alias("Age_filled")
    )
    .select(["Sex","Age","Age_filled"])
    .filter(pl.col("Age").is_null())
    .head()
)

### Exercise 2
We have the following `DataFrame` with 3 columns

In [ ]:
df = pl.DataFrame(
    {
        "a":[10,None,22,1],
        "b":[8,12,19,None],        
        "c":[5,None,19,None],
    }
)

Add a new column with values from column `c`. If `c` is `null` then use the value from column `b` and if `b` is also `null` use the value from column `a`

In [ ]:
(
    df
    <blank>
)

Add a new column `d` with:
- values from column `c`
- if `c` is `null` then use the median of values in `c`
- ensure the dtype is consistent with `c`

## Solutions

### Solution to Exercise 1
Replace `null` values in the `Age` column to have the `median` of the `Age` column

In [ ]:
csv_file = "../data/titanic.csv"
(
    pl.read_csv(csv_file)
    .with_columns(
        pl.col("Age").fill_null(pl.col("Age").median())
    )
    .head(10)
)

Replace `null` values in the `Age` column to have the `median` of the `Age` column **based on whether the passenger is `male` or `female` in the `Sex` column**.

Expand the following cell if you want a hint

In [ ]:
#Hint: in each fill_null call you need to apply a filter to the `Sex` column before you can call median

In [ ]:
csv_file = "../data/titanic.csv"
(
    pl.read_csv(csv_file)
    .with_columns(
        pl.when(
            pl.col("Sex") == "female"
        )
        .then(
            pl.col("Age").fill_null(pl.col("Age").filter(pl.col("Sex") == "female").median())
        )
        .otherwise(
            pl.col("Age").fill_null(pl.col("Age").filter(pl.col("Sex") == "male").median())
        )
        .alias("Age_filled")
    )
    .select(["Sex","Age","Age_filled"])
    .filter(pl.col("Age").is_null())
    .head()
)

### Solution to Exercise 2
We have the following `DataFrame` with 3 columns

In [ ]:
df = pl.DataFrame(
    {
        "a":[10,None,22,1],
        "b":[8,12,19,None],        
        "c":[5,None,19,None],
    }
)

Add a new column `d` with values from column `c`. If `c` is `null` then use the value from column `b` and if `b` is also `null` use the value from column `a`

In [ ]:
(
    df
    .with_columns(
        pl.coalesce(["c","b","a"]).alias("d")
    )
)

Add a new column `d` with:
- values from column `c`
- if `c` is `null` then use the median of values in `c`
- ensure the dtype is consistent with `c`

In [ ]:
(
    df
    .with_columns(
        pl.coalesce(["c",pl.col("c").median()]).cast(pl.Int64).alias("d")
    )
)